# Lab 5 - Adding Crypto

## 5.1 - Primer

This lab will build on your knowledge of general TCP/IP communications and add elements of security using cryptography, provided by the PyNaCl library.

The aim of this lab is to improve the TCP based FTP Client and Server applications you created in Lab 2 by adding elements of security using cryptography. You may begin by either creating a new PyCharm project following the instructions from Lab 2 or by copying the project you created for Lab 2 and modifying it; be sure to start with a working lab 2 implementation.

To provide the cryptography services used for this lab we'll be making use of the PyNaCl cryptography library. PyNaCl is a Python module which makes use of the libsodium crypography library. libsodium is meant to simplify the usage of common cryptography operations in order to prevent common mistakes made in distributed systems. You can learn more about PyNaCl at the [official documentation](https://pynacl.readthedocs.io/en/latest/index.html). The remainder of this section walks through basic cryptographic operations using PyNaCl. Run the code blocks to follow along.

In [1]:
import nacl.secret # Symmetric Encryption 
import nacl.public # Asymmetric Encryption
import nacl.utils # Utilities

### Symmetric Encryption
For Symmetric Encryption we'll be using the very simplified PyNaCl SecretBox class. This class will do all the heavy lifting for us by creating appropriate nonces and authentication codes (digests/hashes) to ensure that our messages aren't tampered with and that we know where they came from.

The PyNaCl SecretBox class will handle the creation of random nonces for us. PyNaCl does not however track these nonces, so we'll be relying on TCP to ensure messages aren't sent out of order or duplicated by accident. 

The worst case from a security perspective is that an attacker doing a man-in-the-middle could insert duplicate data into our TCP connections, which would be processed as if it were a real message. We can overcome this by adding our own sequence numbers to our plaintext messages or adding additional checksums in our protocol, but we'll mitigate this risk by confirming downloads using hashes after they complete. 

Here is how we would generate a symmetric key and use it to encrypt and decrypt data.

In [2]:
#based on PyNaCl examples at https://pynacl.readthedocs.io/en/latest/secret/

#generate a random key of the correct size for our cipher
key = nacl.utils.random(nacl.secret.SecretBox.KEY_SIZE) 

#create a "SecretBox", an abstraction to represent encrypting and decrypting messages
box = nacl.secret.SecretBox(key)

message = b'Some arbitrary message in bytes.'

#encrypt
ciphertext = box.encrypt(message)
#ciphertext is not a bytes object but it is bytes like and can be transmitted over a socket
print(ciphertext)

#note that the ciphertext has a MAC and nonce added and so its length is larger than the message
print(len(ciphertext) > len(message)) # this is true

#decrypt
plaintext = box.decrypt(ciphertext)

print(message == plaintext) # this is also true

b'\x96`\x15\xe4\x1c\xa9\xa3\x1e]\xa4\xefh\xd9v\xd0\xa9\xeb\xf7\x8b\xa6\xaf5$\x0f\xdc\xa3\x10r\n\xa3\xd7\n\xe4\x8e\x90\xa8\xe4\\qE\x02lh4\x83\x15\x86r\x8e\x96\x93^\xc8\xaa"\xba\r\x94\xd3yn\'\xe3\xdc6_U\xa7\x95Q:\x8e'
True
True


### Asymmetric Encryption

Now that we know how to generate random symmetric keys and use them to encrypt and decrypt we need some manner to share them between our distributed systems. Here is an example of asymmetric encryption.

In [5]:
from nacl.public import PrivateKey, PublicKey, Box

#based on PyNaCl examples at https://pynacl.readthedocs.io/en/latest/public/

# Generate a private key.
# This is essentially a Public/Private key pair since it can also derive a public key.
k_priv = PrivateKey.generate() # <-- PyNaCl object

# Encode a Public/Private key pair so it can be stored on disk (hopefully somewhere safe).
# This converts the key object into a bytes-like type.
encoded_k_priv = k_priv.encode()

# Decode the key into a PyNaCl object using the PrivateKey constructor.
k_priv_decoded = PrivateKey(encoded_k_priv)

#get the public portion of the key-pair
k_pub = k_priv_decoded.public_key

print(type(k_priv) == type(k_priv_decoded))

# Assume we have some sort of function that sends our public key as bytes to the other system.
# send_public_key_to_other_system(k_pub.encode())
                                
# Assume we have some sort of function that gets you the other system's encoded public key.
# encoded_others_k_pub = recv_public_key_from_other_system()

# We create a PublicKey object from the received bytes
# others_k_pub = PublicKey(encoded_others_k_pub)

# Create a public key "box" that goes from us to them.
# box = Box(k_priv, others_k_pub)

# Now we can decrypt incoming ciphertext encrypted with others_k_priv
# incomingMessage = box.decrypt(ciphertext)

True


### Hashing

In [6]:
import nacl.hash

#based on PyNaCl examples at https://pynacl.readthedocs.io/en/latest/hashing/

# Hashing a byte array is as simple as calling a hash constructor.
hasher = nacl.hash.sha256
# Taking some long message we can generate a secure digest (digest is umbrella term for hashing).
msg = 16*b'Through the Looking Glass '
digest = hasher(msg)

print(msg)
print(digest)

b'Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass Through the Looking Glass '
b'f0331f94b8daa312c269b9ca4c49132a3396337315959e85ccfc21a58595be5b'


In [7]:
# Consider how we may validate single hash for a file if it is transmitted in fragments.
digest_full = hasher(b'ab')
digest_frag = hasher(b'a' + b'b')

print(digest_full == digest_frag) # This is True

True


## 5.2 - Requirements
* The client and server will communicate via TCP socket using the SecureTCPFileTransfer class. It is recommended that you copy your interface from lab 2 and build on the encryption mechanism.
* Both client and server generate private and public keys on initialization;
* When the client connects to the server they will perform a symmetric key exchange using public key encryption. The server will generate the symmetric key and send it to the client;
* All communications following the key exchange will be encrypted using the shared symmetric key;
* If the server ever receives a ciphertext which does not decrypt to a valid plaintext, it will send an appropriate error message to the client and close the connection;
* Due to the session oriented nature of the cryptographic communications, all communications will be conducted over a single socket which must not be closed and re-opened during the exchange of commands or the transferring of files;
* Files must be transferred as a series of smaller messages containing no more than 1024 bytes of data. 
    * &#128073; The encrypted payload is longer than the unencrypted data due to nonce and hash size - this must be accounted for. Note that the encrypted payload size is constant for a constant plaintext size.
* After a file transfer is complete the sending component will send a hash of the complete file which will be verified by the receiving side, if the hash does not match the receiving side will signal an error which the client will display to the user;

This lab will require adding or modifying code to the FTServer, FTClient, and SecureTCPFileTransfer. The recommended approach is to create a handshake method for the server and for the client first that performs the key exchange. Once this has been verified, move on to encrypting communications in the interface, and finally adding hash checking.

Hint: It is not recommended to use the send/receive command methods in the interface to perform the key exchange as you'll need to add symmetric encryption to these methods. Instead, consider creating interface send and receive methods specifically for unencrypted traffic that are used to perform the key exchange.

Note: As in previous labs, your FTClient and FTServer must be capable of executing even if the directory they are installed in is moved, ie do not hard code absolute paths to the Send and Recieve folders, use relative paths. Marks will be deducted if your code crashes because it was moved to a new location (ie the computer of the instructor marking it.)

## 5.3 - Testing

An important aspect of distributed system development is to properly test your implementation. Be sure to run through a good set of test cases to both test proper functionality and for error cases. The files you transfer should be of different sizes and be able to transmit in either direction. Remember to test your program's ability to handle errors arising from using an incorrect symmetric key.

Explain your testing in your submitted code, and include any test files as needed. Reminder: you don't *need* to include testing information in your submission but if your code fails, clearly documented testing can provide partial marks.

# Questions
1. Why do we choose the FTServer to generate the symmetric key and not the FTClient? For example, what would happen if the key was generated by the FTClient and a malicious Client attempted to connect?

# Submission

Your objective for this lab is to implement the following files using the requirements listed in part 5.2.

1. SecureTCPFileTransfer.py
2. FTClient.py
3. FTServer.py

All files included in this lab (i.e., your project) are to be submitted in a zip file (see lab_report_template.ipynb for instructions). Use the lab report template to reference the files that you modified. Include your answer to the question in the report.
